In [4]:
import pandas as pd
import numpy as np
import re
import json
import time

In [5]:
data = 'cleaned_history_.json'
unprocessed_data = pd.read_json(data)

In [28]:
unprocessed_data.head(5)

,favicon_url,page_transition,title,url,client_id,time_usec,link
0,https://www.vg.no/gfx/icons/apple-touch-icon-1...,TYPED,Nyheter fra Norges mest leste nettavis – VG,https://www.vg.no/,Iiteen/E0kB7QdKeRu1j8Q==,1665346443057756,www.vg.no
1,None,RELOAD,Ny oppskyting fra Kim - USA rådfører seg med a...,https://www.vg.no/nyheter/utenriks/i/kE6V7X/ny...,Iiteen/E0kB7QdKeRu1j8Q==,1665312300407971,www.vg.no
2,https://www.vg.no/cnp-assets/favicon-29bc799f/...,LINK,Zelenskyj med kryptisk Krim-melding – VG,https://www.vg.no/nyheter/utenriks/i/O8rdRw/ze...,Iiteen/E0kB7QdKeRu1j8Q==,1665312249062191,www.vg.no
3,https://www.vg.no/cnp-assets/favicon-29bc799f/...,LINK,Zelenskyj tar Japans parti i «krig» med Russland…,https://www.vg.no/nyheter/utenriks/i/696RlL/ze...,Iiteen/E0kB7QdKeRu1j8Q==,1665312169648724,www.vg.no
4,https://www.vg.no/cnp-assets/favicon-29bc799f/...,LINK,Haaland hylles verden rundt: – Det er brutalt ...,https://www.vg.no/sport/fotball/i/EQae6G/haala...,Iiteen/E0kB7QdKeRu1j8Q==,1665312065516677,www.vg.no


In [6]:
unprocessed_data = unprocessed_data.reset_index()
unprocessed_data = unprocessed_data.rename(columns={"index":"id"})

In [7]:
unprocessed_data
data = [(src, dst) for src, dst in zip(unprocessed_data['id'], unprocessed_data['time_usec'])]

In [96]:
def Sort(sub_li):
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    return(sorted(sub_li, key = lambda x: x[1]))    
  
# Driver Code
print(Sort(data))
data = np.array(Sort(data))

[(149, 1661175088282028), (148, 1661175091402120), (147, 1661175113250506), (146, 1661175188508216), (145, 1661234269630868), (144, 1661234269635491), (143, 1661240414177422), (142, 1661240418438731), (141, 1661243244271721), (140, 1661243244276620), (139, 1661247955097877), (308, 1661247975047118), (309, 1661247975047118), (306, 1661247985480733), (307, 1661247985480733), (304, 1661248076078824), (305, 1661248076078824), (302, 1661248160037859), (303, 1661248160037859), (300, 1661348924082368), (301, 1661348924082368), (138, 1661348924087241), (298, 1661348924087375), (299, 1661348924087375), (322, 1661348929445939), (321, 1661351407094192), (320, 1661351412811720), (319, 1661351415334184), (318, 1661351519563767), (317, 1661355855341375), (316, 1661355855345947), (315, 1661355855346014), (314, 1661355858214755), (313, 1661355858214807), (312, 1661509911022686), (311, 1661509911022740), (296, 1661509924923843), (297, 1661509924923843), (294, 1661509932513894), (295, 1661509932513894),

In [94]:
from datetime import datetime
import time

def show_time(stamp):
    times = stamp / 1000000
    return datetime.utcfromtimestamp(times).strftime("%D %H:%M")
times = 1665312603834859/1000000
datetime.utcfromtimestamp(times).strftime("%D %H:%M")

show_time(1665312603834859-(5*60000000))

'10/09/22 10:45'

In [134]:
# Bins containing lists of data, add to list if within 5 mins of last entry

class SessionString:
    def __init__(self, item_id: int, usec: int, timespan = 5):
        self.id = item_id
        self.usec = usec
        self.timespan = timespan

        self.top = self.timespan * 60000000
        self.bottom = -(self.timespan * 60000000)

        # data sctructure points
        self.before = None
        self.after = None
        

    # This will always be called on head, which is the first object timewise
    def add(self, new_entry: object):
        difference = self.usec - new_entry.usec
        if difference < self.top and difference > self.bottom:
            print(show_time(a))
            print(show_time(b))
            print(f'Within {self.timespan} mins!')
            self.place(new_entry, difference)
            return True
        else:
            print(show_time(a))
            print(show_time(b))
            print('Not Within mins!', difference / 60000000, 'minutes apart.')
            return False


    def place(self, new_entry: object, difference: int):
        head = self
        # if difference is less than 0, set new head.
        if difference < 0:
            # add 
            head.before = new_entry
            new_entry.after = head
            # add difference to bottom (maximum allowed time)
            new_entry.bottom = head.bottom - difference
            self = new_entry

        # if difference is positive, we will not replace head, but we will add to tail.
        else:
            item = head
            before = None
            # go to after untill either item is null (we reached tail)
            # or fix after.
            while item.after:

                if item.usec < new_entry.usec:
                    # if we're at tail:
                    item = item.after
                
                else: 
                    # place
                    before = item.before

                    new_entry.before = before
                    before.after = new_entry
                    new_entry.after = item
                    item.before = new_entry
                    break
            
            # if while loop ran out due to no after:
            if not item.after:
                head.bottom += difference
                item.after = new_entry
                new_entry.before = item

    def as_list(self):
        values = []
        # go to first item
        head = self.get_head()
        
        # iterate and grab all entries
        while head.after:
            values.append((head.id, head.usec))
            head = head.after

        return values
        
    def get_head(self):
        head = self
        while head.before:
            head = head.before
        return head


In [135]:
sessions = []
for entry in data:
    i = 0
    # cold start
    if len(sessions) == 0:
        entry = SessionString(item_id=entry[0], usec=entry[1])
        sessions.append([entry])
    else:
        entry = SessionString(item_id=entry[0], usec=entry[1])
        print(sessions)
        head = sessions[i][0]
        while not head.add(entry):
            i += 1

[[<__main__.SessionString object at 0x16988f0d0>]]


AttributeError: 'list' object has no attribute 'add'

In [101]:
sessions = []
len(sessions)

1

In [124]:
a = 1664623909207491
b = 1664623909207491
print(show_time(a))
print(show_time(b))
difference = a - b
if difference < (5 * 60000000) and difference > -(5 * 60000000):
    print(show_time(a))
    print(show_time(b))
    print('Within 5 mins!')
else:
    print(show_time(a))
    print(show_time(b))
    print('Not Within mins!', difference / 60000000, 'minutes apart.')


10/01/22 11:31
10/01/22 11:31
10/01/22 11:31
10/01/22 11:31
Within 5 mins!


In [8]:
# Bins containing lists of data, add to list if within 5 mins of last entry
import pandas as pd
import numpy as np
import re
import json
import time
from datetime import datetime

class SessionString:
    def __init__(self, item_id: int, usec: int, timespan = 5):
        self.id = item_id
        self.usec = usec
        self.timespan = timespan

        self.top = self.timespan * 60000000
        self.bottom = -(self.timespan * 60000000)

        # data sctructure points
        self.before = None
        self.after = None

    def __show_time(self, stamp):
        times = stamp / 1000000
        return datetime.utcfromtimestamp(times).strftime("%D %H:%M")
        

    # This will always be called on head, which is the first object timewise
    def add(self, new_entry: object):
        difference = self.usec - new_entry.usec
        if difference < self.top and difference > self.bottom:
            # print(f'Within {self.timespan} mins! at {difference/60000000}')
            self.place(new_entry, difference)
            return True
        else:
            # print('Not Within mins!', difference / 60000000, 'minutes apart.')
            return False


    def place(self, new_entry: object, difference: int):
        head = self
        # if difference is less than 0, set new head.
        if difference < 0:
            # add 
            head.before = new_entry

            new_entry.after = head
            new_entry.before = None
            # add difference to bottom (maximum allowed time)
            new_entry.bottom = head.bottom - difference
            self = new_entry

        # if difference is positive, we will not replace head, but we will add to tail.
        else:
            head = self
            # go to after untill either item is null (we reached tail)
            # or fix after.
            if head.after:
                item = head
                while item.after:
                    if item.usec < new_entry.usec:
                        # if we're at tail:
                        item = item.after
                    
                    else: 
                        # place                        
                        before = item
                        new_entry.before = before
                        before.after = new_entry
                        new_entry.after = item
                        item.before = new_entry
                        
                        break
            
            # if while loop ran out due to no after:
            elif not head.after:
                head.bottom += difference
                head.after = new_entry
                new_entry.before = head
                new_entry.after = None
                self = new_entry

    def as_list(self):
        values = []
        # iterate and grab all entries
        head = self

        if head.after:
            while head.after:
                print('Found After')
                values.append((head.id, head.usec))
                head = head.after
        else: 
            print('no head')
            values.append((head.id, head.usec))

        yield values
        
    def get_head(self):
        head = self
        while head.before:
            head = head.before
        return head

data = 'cleaned_history_.json'
unprocessed_data = pd.read_json(data)

unprocessed_data = unprocessed_data.reset_index()
unprocessed_data = unprocessed_data.rename(columns={"index":"id"})

data = [(src, dst) for src, dst in zip(unprocessed_data['id'], unprocessed_data['time_usec'])]

sessions = []
for entry in data:
    i = 0
    # cold start
    if len(sessions) == 0:
        entry = SessionString(item_id=entry[0], usec=entry[1])
        sessions.append(entry)
    else:
        entry = SessionString(item_id=entry[0], usec=entry[1])
        head = sessions[i]
        while not head.add(entry):
            i += 1
            if i + 1 > len(sessions):
                sessions.append(entry)
                break
            else:
                head = sessions[i]

#for entry in sessions:
#    print([x for x in entry.as_list()])


In [72]:
class SessionNode:
    def __init__(self, item_id: int, usec: int):
        self.id = item_id
        self.usec = usec

        self.before = None
        self.after = None

    def get_data(self):
        return (self.id, self.usec)
      
    def set_data(self, item_id, usec):
        self.id = item_id
        self.usec = usec
        
    def get_after(self):
        return self.after

    def set_after(self, after):
        self.after = after
      

    def get_before(self):
        return self.before
 
    def set_before(self, before):
        self.before = before



In [82]:
class SessionList:
    def __init__(self, head = None, tail = None, timespan = 5):
        self.head = head
        self.tail = None
        self.timespan = timespan

        self.top = self.timespan * 60000000
        self.bottom = -(self.timespan * 60000000)
    
        self.count = 0

    # takes tupe (id, usec)
    def insert(self, data):
        new_node = SessionNode(data[0], data[1])

        if not self.head:
            self.head = new_node
            self.count += 1
            print('Head')
            return True

        else:
            difference = self.head.usec - new_node.usec
            print(self.top, self.bottom, difference)
            print(self.head.id, new_node.id)
            print(new_node.usec, 'NODE')
            print(self.head.usec, 'HEAD')
            print(difference, 'DIFFERENCE \n')
            if difference < self.top and difference > self.bottom: # DIFFERENCE = 0 ?????
                print('Accepted \n')
                self.add(new_node)
                self.count += 1

                if difference > 0:
                    self.top += difference
                else:
                    self.bottom -= difference
                return True
            else:
                print('Rejected \n')
                return False

    def add(self, new_node):
        if self.head.usec < new_node.usec:
            self.head.set_after(new_node)
        else:
            new_node.set_after(self.head)
            self.head = new_node
        return

    def show(self):
        values = []
        item = self.head
        while item.get_after():
            values.append(item.get_data())
            item = item.get_after()
        # as while loop above will not append the tail node, we add:
        values.append(item.get_data())
        return values

In [92]:
data = [(src, dst) for src, dst in zip(unprocessed_data['id'], unprocessed_data['time_usec'])]

sessions = []
for entry in data:
    i = 0
    # cold start
    if len(sessions) == 0:
        data_list = SessionList()
        data_list.insert(entry)
        sessions.append(data_list)
    else:
        data_list = sessions[i]
        while not data_list.insert(entry): # While im False
            i += 1
            if i + 1 > len(sessions): # End of line, create new.
                data_list = SessionList()
                data_list.insert(entry)
                sessions.append(data_list)
                break
            else:
                data_list = sessions[i]

        # data_list = SessionString(item_id=entry[0], usec=entry[1])
        # head = sessions[i]
        # while not head.add(entry):


    

Head
300000000 -300000000 34142649785
0 1
1665312300407971 NODE
1665346443057756 HEAD
34142649785 DIFFERENCE 

Rejected 

Head
300000000 -300000000 34193995565
0 2
1665312249062191 NODE
1665346443057756 HEAD
34193995565 DIFFERENCE 

Rejected 

300000000 -300000000 51345780
1 2
1665312249062191 NODE
1665312300407971 HEAD
51345780 DIFFERENCE 

Accepted 

300000000 -300000000 34273409032
0 3
1665312169648724 NODE
1665346443057756 HEAD
34273409032 DIFFERENCE 

Rejected 

351345780 -300000000 79413467
2 3
1665312169648724 NODE
1665312249062191 HEAD
79413467 DIFFERENCE 

Accepted 

300000000 -300000000 34377541079
0 4
1665312065516677 NODE
1665346443057756 HEAD
34377541079 DIFFERENCE 

Rejected 

430759247 -300000000 104132047
3 4
1665312065516677 NODE
1665312169648724 HEAD
104132047 DIFFERENCE 

Accepted 

300000000 -300000000 34466280165
0 5
1665311976777591 NODE
1665346443057756 HEAD
34466280165 DIFFERENCE 

Rejected 

534891294 -300000000 88739086
4 5
1665311976777591 NODE
16653120655166

In [101]:
for l in sessions:
    print(l.show())

[(153, 1665346420035831), (152, 1665346420035831), (151, 1665346420056391), (150, 1665346420056391), (0, 1665346443057756)]
[(9, 1665311367101901), (8, 1665311393743599), (7, 1665311635680594), (6, 1665311778883343), (5, 1665311976777591), (4, 1665312065516677), (3, 1665312169648724), (2, 1665312249062191), (1, 1665312300407971)]
[(11, 1665307957601075), (10, 1665307968662225)]
[(173, 1665307280880964), (172, 1665307280880964), (13, 1665307280902240), (171, 1665307280902307)]
[(23, 1665220264376649), (22, 1665220271976214), (21, 1665220296205634), (20, 1665220358709719), (19, 1665220391586616), (18, 1665220447504603), (17, 1665220501919738), (16, 1665220518155552), (15, 1665220914999309), (14, 1665221130265615)]
[(24, 1665042311649973)]
[(27, 1664791704028569), (191, 1664791704028642)]
[(195, 1664627631208558), (194, 1664627631208558), (30, 1664627631212237), (193, 1664627631212319)]
[(201, 1664623909204265), (200, 1664623909204265), (33, 1664623909207412), (199, 1664623909207491)]
[(3